In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import time
import random
import csv
from ast import literal_eval

In [2]:
df1 = pd.read_csv('craigslist.csv')
df1.head(2)

,url,post_id,post_date,description,num_beds,num_baths,price,address,sqft,features
0,https://sandiego.craigslist.org/csd/apa/d/san-...,7409829048,2021-11-18T15:35:15-0800,4 weeks free on 2 bedrooms4 weeks free on 1 be...,2,2,3395,1302 Frankfort St.,-1.0,"['EV charging', 'air conditioning', 'cats are ..."
1,https://sandiego.craigslist.org/csd/apa/d/san-...,7401811487,2021-10-31T15:50:52-0700,Bedrooms: 0Bathrooms: 1Square Feet: 620Address...,0,1,2252,7777 Westside Dr Suite 118 near Mission Center...,620.0,"['cats are OK - purrr', 'dogs are OK - wooof',..."


In [3]:
df2 = pd.read_csv('scrapped craigslist post.csv')
df2['description'] = df2['description'].apply(lambda x: x.strip('QR Code Link to This Post\n\n\n'))
df2['info'] = df2['info'].apply(lambda x: literal_eval(x))
df2.head(2)

,Unnamed: 0,post_id,post_date,description,info,features
0,0,7406258566,2021-11-10T14:23:12-0800,OINT LOMA BAY APARTMENTS\n\nWE OFFER QUALIFYIN...,"[1, 1, 1940, 2449 SOTO ST]","[<span>EV charging</span>, <span>cats are OK -..."
1,1,7402763878,2021-11-02T17:49:42-0700,Bedrooms: 3Bathrooms: 2Square Feet: 1355Addres...,"[3, 2, 3911, 7777 Westside Drive]","[<span>apartment</span>, <span>w/d in unit</sp..."


In [4]:
df2 = df2.assign(
    num_beds = df2['info'].apply(lambda x: x[0]),
    num_baths = df2['info'].apply(lambda x: x[1]),
    price = df2['info'].apply(lambda x: x[2]),
    address = df2['info'].apply(lambda x: x[3])
)
df2 = df2.drop(columns = ['info', 'Unnamed: 0'])
df2.head(3)

,post_id,post_date,description,features,num_beds,num_baths,price,address
0,7406258566,2021-11-10T14:23:12-0800,OINT LOMA BAY APARTMENTS\n\nWE OFFER QUALIFYIN...,"[<span>EV charging</span>, <span>cats are OK -...",1,1,1940,2449 SOTO ST
1,7402763878,2021-11-02T17:49:42-0700,Bedrooms: 3Bathrooms: 2Square Feet: 1355Addres...,"[<span>apartment</span>, <span>w/d in unit</sp...",3,2,3911,7777 Westside Drive
2,7405170179,2021-11-08T09:03:56-0800,Elan Beachhouse Del Mar\nProperty Address\n\n2...,[<span>application fee details: <b>$35.00</b><...,1,1,2995,2515 Camino del Mar


In [5]:
test_x = df2['features'].apply(
    lambda x: [re.search('>(.*)<', y) for y in x.split(',')]
)
test_x2 = test_x.apply(
    lambda x: [y.group(1) for y in x if y is not None]
)

In [6]:
test_x2

0       [EV charging, cats are OK - purrr, dogs are OK...
1                       [apartment, w/d in unit, carport]
2       [application fee details: <b>$35.00</b>, cats ...
3       [cats are OK - purrr, dogs are OK - wooof, apa...
4       [EV charging, cats are OK - purrr, dogs are OK...
                              ...                        
1292    [cats are OK - purrr, flooring: <b>carpet</b>,...
1293    [application fee details: <b>$40/adult applica...
1294            [apartment, w/d in unit, detached garage]
1295    [air conditioning, flooring: <b>wood</b>, apar...
1296                                  [monday 2021-11-01]
Name: features, Length: 1297, dtype: object

In [7]:
df2 = df2.assign(features = test_x2)

In [8]:
df2

,post_id,post_date,description,features,num_beds,num_baths,price,address
0,7406258566,2021-11-10T14:23:12-0800,OINT LOMA BAY APARTMENTS\n\nWE OFFER QUALIFYIN...,"[EV charging, cats are OK - purrr, dogs are OK...",1,1,1940,2449 SOTO ST
1,7402763878,2021-11-02T17:49:42-0700,Bedrooms: 3Bathrooms: 2Square Feet: 1355Addres...,"[apartment, w/d in unit, carport]",3,2,3911,7777 Westside Drive
2,7405170179,2021-11-08T09:03:56-0800,Elan Beachhouse Del Mar\nProperty Address\n\n2...,"[application fee details: <b>$35.00</b>, cats ...",1,1,2995,2515 Camino del Mar
3,7406281281,2021-11-10T15:17:02-0800,We are open for business! To help us continue ...,"[cats are OK - purrr, dogs are OK - wooof, apa...",1,1,2858,"4855 Ariva Way, #130"
4,7405635647,2021-11-09T09:06:20-0800,Encompass yourself with lush landscaping and c...,"[EV charging, cats are OK - purrr, dogs are OK...",2,2,3535,5280 Fiore Terrace
...,...,...,...,...,...,...,...,...
1292,7403655906,2021-11-04T17:10:53-0700,2 Bd. 2 Bath (upstairs)&(downstairs)Sq ft. \n...,"[cats are OK - purrr, flooring: <b>carpet</b>,...",2,2,1800,4065 iowa st. near El Cajon Blvd.
1293,7405944950,2021-11-09T23:37:17-0800,pectacular 1-bedroom/1-bathroom units have bee...,[application fee details: <b>$40/adult applica...,1,1,2695,Kellogg St near Rosecrans
1294,7402527487,2021-11-02T09:47:51-0700,San Diego's most popular destinations and stun...,"[apartment, w/d in unit, detached garage]",2,2,3819,5395 Napa St
1295,7405332384,2021-11-08T13:48:34-0800,SDHC affordable housing program third floor 1B...,"[air conditioning, flooring: <b>wood</b>, apar...",1,1,1500,Washington St


In [24]:
fieldnames = [
    'url',
    'post_id',
    'post_date',
    'description',
    'num_beds',
    'num_baths',
    'price',
    'address',
    'sqft',
    'features',
    'scrape_date'
]

In [26]:
df3 = pd.concat([df2,df1])
df3 = df3.reset_index(drop = True)
df3 = df3.assign(scrape_date = df3['post_date'])
df3 = df3[fieldnames]

In [29]:
df3.to_csv('craigslist.csv', index = False)